## Prefer Helper Classes Over Bookkeeping with Dictionaries and Tuples

In [1]:
class SimpleGradebook(object):
    def __init__(self):
        self._grades = {}
    
    def add_student(self, name):
        self._grades[name] = []

    def report_grade(self, name, score):
        self._grades[name].append(score)

    def average_grade(self, name):
        grades = self._grades[name]
        return sum(grades) / len(grades)

book = SimpleGradebook()
book.add_student('Isaac Newton')
book.report_grade('Isaac Newton', 90)
print(book.average_grade('Isaac Newton'))

90


In [4]:
# refactoring as classes
import collections
Grade = collections.namedtuple('Grade', ('score', 'weight'))

# as an example
grade = Grade(10, 0.25)
grade

Grade(score=10, weight=0.25)

In [7]:
class Subject(object):
    def __init__(self):
        self._grades = []
        
    def report_grade(self, score, weight):
        self._grades.append(Grade(score, weight))

    def average_grade(self):
        total, total_weight = 0, 0
        for grade in self._grades:
            total += grade.score * grade.weight
            total_weight += grade.weight
        return total / total_weight

subject = Subject()
subject.report_grade(10, 0.5)
subject.report_grade(8, 0.5)
subject.average_grade()

9.0

In [9]:
class Student(object):
    def __init__(self):
        self._subjects = {}
        
    def subject(self, name):
        if name not in self._subjects:
            self._subjects[name] = Subject()
        return self._subjects[name]
    
    def average_grade(self):
        total, count = 0, 0
        for subject in self._subjects.values():
            total += subject.average_grade()
            count += 1
        return total / count
    

In [10]:
class Gradebook(object):
    pass
    def __init__(self):
        self._students = {}

    def student(self, name):
        if name not in self._students:
            self._students[name] = Student()
        return self._students[name]
    

In [13]:
book = Gradebook()
albert = book.student('Albert Einstein')
math = albert.subject('Math')
math.report_grade(80, 0.50)
math.report_grade(100, 0.50)

print(albert.average_grade())

90.0


## Accept functions for simple interfaces instead of classes

In [1]:
names = ["Put", "some", "names", "here"]
names.sort(key=lambda x: len(x))
names

['Put', 'some', 'here', 'names']

Here we have a defaultdict example where a hook logs missing keys:

In [2]:
def log_missing():
    return 0

In [4]:
current = {"green": 12, "blue": 3}
increments = [
    ("red", 5),
    ("blue", 17),
    ("orange", 9)
]

In [9]:
# https://docs.python.org/2/library/collections.html#collections.defaultdict
import collections
result = collections.defaultdict(log_missing, current)
result

defaultdict(<function __main__.log_missing>, {'blue': 3, 'green': 12})

In [10]:
for key, value in increments:
    result[key] += value
result

defaultdict(<function __main__.log_missing>,
            {'blue': 20, 'green': 12, 'orange': 9, 'red': 5})

Python allows classes to define the __call__ special method which allows the object to be called just like a function.

In [14]:
class SomeCounter(object):
    def __init__(self):
        self.added = 0
        
    def __call__(self):
        self.added += 1
        return 0
    
counter = SomeCounter()
counter.added

0

In [15]:
counter()
counter.added

1

In [17]:
callable(counter)

True

## Use @classmethod polymorphism to construct objects generically.
Python only supports a single constructor per class via the init method. We can use @classmethodd to define alternative constructors.

In [6]:
# https://stackoverflow.com/questions/682504/what-is-a-clean-pythonic-way-to-have-multiple-constructors-in-python
from random import randint

class Cheese(object):
    def __init__(self, num_holes=0):
        ''' defaults to a solid cheese '''
        self.number_of_holes = num_holes

    @classmethod
    def random(cls):
        return cls(randint(0, 100))

    @classmethod
    def slightly_holey(cls):
        return cls(randint(0, 33))
                   
    @classmethod
    def very_holey(cls):
        return cls(randint(66, 100))
                   
gouda = Cheese()
emmentaler = Cheese.random()
leerdammer = Cheese.slightly_holey()

In [7]:
gouda.number_of_holes

0

In [8]:
emmentaler.number_of_holes

77

In [9]:
leerdammer.number_of_holes

3

## Initialise parent classes with super
Here is the old way by directly calling the parent classes init method.

In [11]:
class MyBaseClass(object):
    def __init__(self, value):
        self.value = value
        
class MyChildClass(MyBaseClass):
    def __init__(self):
        MyBaseClass.__init__(self, 5)

child = MyChildClass()
child.value

5

Calling the init method directly can lead to unexpected behavior.

In [12]:
class TimesTwo(object):
    def __init__(self):
        self.value *= 2
        
class PlusFive(object):
    def __init__(self):
        self.value += 5
        
# multiple interitance is something to be avoided in general (see below)
class OneWay(MyBaseClass, TimesTwo, PlusFive):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        TimesTwo.__init__(self)
        PlusFive.__init__(self)
        
oneWay = OneWay(5)
oneWay.value # (5 * 2) + 5

15

In [16]:
class AnotherWay(MyBaseClass, PlusFive, TimesTwo): # order changed here
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        TimesTwo.__init__(self)
        PlusFive.__init__(self)
        
anotherWay = AnotherWay(5)
anotherWay.value # still 15

15

Diamond inheritance causes the common superclasses's __init__ method to run multiple times.

In [18]:
class TimesFive(MyBaseClass):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        self.value *= 5
        
class PlusTwo(MyBaseClass):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        self.value += 2
        
class ThisWay(TimesFive, PlusTwo):
    def __init__(self, value):
        TimesFive.__init__(self, value)
        PlusTwo.__init__(self, value)
        
thisWay = ThisWay(5)
thisWay.value # (5 * 5) + 2 = 27, but this comes back as 7 = 5 + 2 (value is reset to 5 when the parent class init called)

7

Python 2.2 addded the super built in function and defined the method resolution order (MRO). The MRO standardizes superclasses depth first, left to right. It also ensures that common superclasses in diamond hierarchies are only run once. 

In [25]:
class TimesFiveCorrect(MyBaseClass):
    def __init__(self, value):
        super(TimesFiveCorrect, self).__init__(value) # in Python 3 you don't need to include the class name in super
        self.value *= 5
        
class PlusTwoCorrect(MyBaseClass):
    def __init__(self, value):
        super(PlusTwoCorrect, self).__init__(value)
        self.value += 2
        
class GoodWay(TimesFiveCorrect, PlusTwoCorrect):
    def __init__(self, value):
        super(GoodWay, self).__init__(value)
        
goodWay = GoodWay(5)
goodWay.value 

35

In [26]:
GoodWay.mro() # get the method resolution order

[__main__.GoodWay,
 __main__.TimesFiveCorrect,
 __main__.PlusTwoCorrect,
 __main__.MyBaseClass,
 object]

In [27]:
(5 + 2) * 5

35

https://www.python.org/download/releases/2.3/mro/

## Use multiple inheritance only for mix-in utility classes
Its better to avoid multiple inheritance altogether and writing a min-in instead. A mix-in is a small class that only defines a set of additional methods. 

In [28]:
# an example from https://andrewbrookins.com/technology/mixins-in-python-and-ruby-compared/

class WalkerMixin:
    @property
    def max_speed(self):
        return 1


class RunnerMixin:
    @property
    def max_speed(self):
        return 4


class FlierMixin:
    @property
    def max_speed(self):
        return 10


class SortaFastHero(RunnerMixin):
    """This hero can run, which is better than walking."""
    pass


class CuriouslySlowHero(WalkerMixin, FlierMixin):
    """The order of this class's parents made her curiously slow!"""
    pass


class FastestHero(FlierMixin, RunnerMixin):
    """The fastest hero can fly, of course."""
    pass


class Board:
    """An imaginary game board that doesn't do anything."""

    def move(self, piece, actions_spent):
        """Move a piece on the board.

        ``piece`` should be movable
        ``actions_spent`` is the number of actions taken to move

        """
        # Fictitious piece-moving machinery here
        return actions_spent * piece.max_speed


def main():
    board = Board()
    rows = (
        ('Hero', 'Total spaces moved'),
        ('a sorta fast hero', board.move(SortaFastHero(), 2)),
        ('a curiously slow hero', board.move(CuriouslySlowHero(), 2)),
        ('the fastest hero', board.move(FastestHero(), 2))
    )
    print('\n')
    for description, movement in rows:
        print("\t{:<22} {:>25}".format(description, movement))


if __name__ == '__main__':
    main()



	Hero                          Total spaces moved
	a sorta fast hero                              8
	a curiously slow hero                          2
	the fastest hero                              20


## Prefer public attributes over private ones
Public attributes can be accessed by anyone using the dot operator on the object. Private fields are specified by prefixing an attribute's name with a double underscore. They can be accessed directly by methods of the containing class.

In [1]:
class MyObject(object):
    def __init__(self):
        self.public_field = 5
        self.__private_field = 10
        
    def get_private_field(self):
        return self.__private_field
    
foo = MyObject()

In [2]:
foo.public_field

5

In [4]:
foo.get_private_field()

10

Directly accessing private fields from outside the class raises an exception.

In [6]:
try:
    foo.__private_field
except Exception as e:
    print(e)

'MyObject' object has no attribute '__private_field'


In [7]:
dir(foo)

['_MyObject__private_field',
 '__class__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__format__',
 '__getattribute__',
 '__hash__',
 '__init__',
 '__module__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'get_private_field',
 'public_field']

In [8]:
foo._MyObject__private_field # we can get these values anyway

10

The syntax for private access does not enforce strict visibility. The motto of Python is that "We are all consenting adults here" and believe that the benefits of being open outweigh the downsides of being closed. To minimize the damage of accessing internals unknowingly, follow the PEP 8 Style Guide: Fields prefixed by a single underscore (like _protected_field) are protected, meaning external users should proceed with caution. Only consider using private attributes to avoid naming conflicts with subclasses that are out of your control. 

## Inherit from collections.abc for custom container types
In the general case you can use ABCMeta with abstractmethod to ensure that child classes implement certain methods.

In [14]:
from abc import ABCMeta, abstractmethod

class AbstractClass(object):
    __metaclass__ = ABCMeta
    
    @abstractmethod
    def someMethod(self):
        raise NotImplementedError("Should implement someMethod()")
      
    
class Child(AbstractClass):
    pass

try:
    Child()
except Exception as e:
    print(e)

Can't instantiate abstract class Child with abstract methods someMethod


The built-in collections.abc module defines a set of abstract base classes that provide all of the typical methods for each container type. 

In [22]:
# https://docs.python.org/3/library/collections.abc.html
# New in version 3.3: Formerly, this module was part of the collections module.
import sys
if sys.version_info.major == 3:
    from collections.abc import Sequence

    class BadType(Sequence):
        pass

    try:
        BadType()
    except Exception as e:
        print(e)

else:
    print("This example was intended for Python 3!")

This example was intended for Python 3!


Inherit directly from Python's container types (like list or dict) for simple use cases. Have your custom container types inherit from the interfaces defined in collections.abc to ensure that your classes match required interfaces and behaviors.